In [1]:
from src.kr6 import KR6
import modern_robotics as mr
import numpy as np
from tpk4170.visualization import Viewer
from tpk4170.models import Grid, Axes
from src.dh import DHFrame
import src.tools as tools

In [2]:
viewer = Viewer(background="white")
viewer.add(Grid())

config1 = np.array([0,-np.pi/2,np.pi/2,0,np.pi,np.pi/3])
config2 = np.array([0,-np.pi/2,np.pi/2,0,0,0])
config3 = np.array([0,0,np.pi/2,0,0,0])

kr6 = KR6(theta=config2)
# kr6.add_FK_to_viewer(viewer)

print(f"Number of iteration: {kr6.IK[3]}")
print(np.allclose(kr6.Tsb_dh, kr6.Tsb_IK, 0.01, 0.01))

kr6.add_nIK_to_viewer(viewer)



Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

Number of iteration: 3
True
